In [1]:
from pykiwoom.kiwoom import *
import time
import pandas as pd

# 로그인
kiwoom = Kiwoom()
kiwoom.CommConnect(block=True)

In [2]:
import sqlite3

# SQLite 데이터베이스 연결
conn = sqlite3.connect('20240715_90%이상 주장 단타.db')

# track_stock 테이블에서 code 열의 모든 값을 불러오기
query = "SELECT code FROM tracked_stocks"
stock_codes_df = pd.read_sql_query(query, conn)
stock_codes = stock_codes_df['code'].tolist()

# 데이터베이스 연결 종료
conn.close()

# 결과를 저장할 빈 데이터프레임 생성
all_data = pd.DataFrame()

# 각 종목 코드에 대해 데이터 요청
for code in stock_codes:
    df = kiwoom.block_request("opt10080",
                              종목코드=code,
                              틱범위=1,
                              output="주식일봉차트조회",
                              next=0)
    df['종목코드'] = code  # 결과 데이터프레임에 종목 코드 추가
    all_data = pd.concat([all_data, df], ignore_index=True)
    time.sleep(1)  # 키움증권 API 요청 제한을 고려한 딜레이

{'trcode': 'opt10080', 'input': [{'주식분봉차트조회요청': ['종목코드', '틱범위', '수정주가구분']}], 'output': [{'주식분차트': ['종목코드']}, {'주식분봉차트조회': ['현재가', '거래량', '체결시간', '시가', '고가', '저가', '수정주가구분', '수정비율', '대업종구분', '소업종구분', '종목정보', '수정주가이벤트', '전일종가']}]}
{'trcode': 'opt10080', 'input': [{'주식분봉차트조회요청': ['종목코드', '틱범위', '수정주가구분']}], 'output': [{'주식분차트': ['종목코드']}, {'주식분봉차트조회': ['현재가', '거래량', '체결시간', '시가', '고가', '저가', '수정주가구분', '수정비율', '대업종구분', '소업종구분', '종목정보', '수정주가이벤트', '전일종가']}]}
{'trcode': 'opt10080', 'input': [{'주식분봉차트조회요청': ['종목코드', '틱범위', '수정주가구분']}], 'output': [{'주식분차트': ['종목코드']}, {'주식분봉차트조회': ['현재가', '거래량', '체결시간', '시가', '고가', '저가', '수정주가구분', '수정비율', '대업종구분', '소업종구분', '종목정보', '수정주가이벤트', '전일종가']}]}
{'trcode': 'opt10080', 'input': [{'주식분봉차트조회요청': ['종목코드', '틱범위', '수정주가구분']}], 'output': [{'주식분차트': ['종목코드']}, {'주식분봉차트조회': ['현재가', '거래량', '체결시간', '시가', '고가', '저가', '수정주가구분', '수정비율', '대업종구분', '소업종구분', '종목정보', '수정주가이벤트', '전일종가']}]}
{'trcode': 'opt10080', 'input': [{'주식분봉차트조회요청': ['종목코드', '틱범위', '수정주가구분']}], 'output

In [3]:
all_data

,현재가,거래량,체결시간,시가,고가,저가,수정주가구분,수정비율,대업종구분,소업종구분,종목정보,수정주가이벤트,전일종가,종목코드
0,+15000,34183,20240715153000,+15000,+15000,+15000,,,,,,,,001750
1,+15050,8281,20240715151900,+15030,+15060,+14990,,,,,,,,001750
2,+15020,7084,20240715151800,+15010,+15030,+14990,,,,,,,,001750
3,+15010,7096,20240715151700,+15010,+15030,+15000,,,,,,,,001750
4,+15010,1176,20240715151600,+15060,+15060,+15010,,,,,,,,001750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21595,-16500,1346,20240711130800,-16520,-16520,-16500,,,,,,,,449450
21596,-16525,30,20240711130700,-16530,-16530,-16525,,,,,,,,449450
21597,-16520,919,20240711130600,-16530,-16530,-16520,,,,,,,,449450
21598,-16530,127,20240711130500,-16525,-16530,-16525,,,,,,,,449450


In [4]:
# 원하는 열만 선택하여 출력
selected_columns = ['현재가', '거래량', '체결시간', '시가', '고가', '저가', '종목코드']
filtered_df = all_data[selected_columns]

In [5]:
# 열 이름 변경 및 순서 재정렬
all_data.rename(columns={
    '현재가': 'close',
    '거래량': 'volume',
    '체결시간': 'date',
    '시가': 'open',
    '고가': 'high',
    '저가': 'low',
    '종목코드' : 'stock_code'
}, inplace=True)

In [6]:
# 원하는 순서로 열 재정렬
reordered_columns = ['stock_code', 'date', 'open', 'high', 'close', 'low', 'volume']
reordered_df = all_data[reordered_columns]

In [7]:
reordered_df

,stock_code,date,open,high,close,low,volume
0,001750,20240715153000,+15000,+15000,+15000,+15000,34183
1,001750,20240715151900,+15030,+15060,+15050,+14990,8281
2,001750,20240715151800,+15010,+15030,+15020,+14990,7084
3,001750,20240715151700,+15010,+15030,+15010,+15000,7096
4,001750,20240715151600,+15060,+15060,+15010,+15010,1176
...,...,...,...,...,...,...,...
21595,449450,20240711130800,-16520,-16520,-16500,-16500,1346
21596,449450,20240711130700,-16530,-16530,-16525,-16525,30
21597,449450,20240711130600,-16530,-16530,-16520,-16520,919
21598,449450,20240711130500,-16525,-16530,-16530,-16525,127


In [8]:
# 체결시간 형식 변환 함수 정의
def format_datetime(dt_str):
    return pd.to_datetime(dt_str, format='%Y%m%d%H%M%S').strftime('%Y-%m-%d %H:%M:%S')

# 체결시간 열 형식 변환
reordered_df['date'] = reordered_df['date'].apply(format_datetime)

C:\Users\euphoria\AppData\Local\Temp\ipykernel_10568\2199826333.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reordered_df['date'] = reordered_df['date'].apply(format_datetime)


In [9]:
reordered_df

,stock_code,date,open,high,close,low,volume
0,001750,2024-07-15 15:30:00,+15000,+15000,+15000,+15000,34183
1,001750,2024-07-15 15:19:00,+15030,+15060,+15050,+14990,8281
2,001750,2024-07-15 15:18:00,+15010,+15030,+15020,+14990,7084
3,001750,2024-07-15 15:17:00,+15010,+15030,+15010,+15000,7096
4,001750,2024-07-15 15:16:00,+15060,+15060,+15010,+15010,1176
...,...,...,...,...,...,...,...
21595,449450,2024-07-11 13:08:00,-16520,-16520,-16500,-16500,1346
21596,449450,2024-07-11 13:07:00,-16530,-16530,-16525,-16525,30
21597,449450,2024-07-11 13:06:00,-16530,-16530,-16520,-16520,919
21598,449450,2024-07-11 13:05:00,-16525,-16530,-16530,-16525,127


In [10]:
# reordered_df 데이터프레임을 stock_code 별로 분리
grouped_data = reordered_df.groupby('stock_code')

# 각 그룹(종목코드별 데이터)을 딕셔너리에 저장
stock_data_dict = {stock_code: data for stock_code, data in grouped_data}

# stock_data_dict에 있는 모든 종목코드의 데이터를 과거 순서로 재배열
for stock_code in stock_data_dict.keys():
    stock_data_dict[stock_code] = stock_data_dict[stock_code].sort_values(by='date')
    stock_data_dict[stock_code][['open', 'high', 'close', 'low', 'volume']] = stock_data_dict[stock_code][['open', 'high', 'close', 'low', 'volume']].apply(pd.to_numeric, errors='coerce').abs()

In [12]:
# # 종목코드 별로 필터링된 데이터를 CSV 파일로 저장
# for stock_code, data in stock_data_dict.items():
#     # date, open, high, close, low, volume 열만 선택하여 저장
#     data[['date', 'open', 'high', 'close', 'low', 'volume']].to_csv(f"{stock_code}.csv", index=False)

In [14]:
import sqlite3

# SQLite 데이터베이스 연결 생성 (데이터베이스가 없으면 새로 생성됩니다)
conn = sqlite3.connect('20240715_90%이상 주장 단타.db')

# 종목코드 별로 필터링된 데이터를 SQLite 데이터베이스에 저장
for stock_code, data in stock_data_dict.items():
    # date, open, high, close, low, volume 열만 선택하여 테이블에 저장
    data[['date', 'open', 'high', 'close', 'low', 'volume']].to_sql(stock_code, conn, if_exists='replace', index=False)

# 데이터베이스 연결 종료
conn.close()